# League (Combined) + Cyberbullying Transformer Model (RoBERTa)

## Initialization

Necessary installations (can be skipped if already done):

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 47.4 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40

Initialization:

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


## Data Prep

In [3]:
def read_league_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["message","toxicity_label"]]

league_data = read_league_data('dataToxic.csv')
league_data = league_data.rename(columns={"toxicity_label": "label"})
league_data['message'] = league_data['message'].str.replace(',',' ',regex=False)
league_data['label'] = league_data['label'].apply(lambda x: 1 if x == 'toxic' else 0)
league_data

,message,label
0,report for unskilled player is useless thx <3 ...,0
1,mimimi,0
2,im comming for you riven pfft focus Zed always...,1
3,thx top no flash for what ? he has 2 kill in l...,1
4,IIII ISI K udyr top dnt us see it? CAMP MORE P...,0
...,...,...
88083,gj &gt;&lt; xD i said ss gj stop go alone plz ...,1
88084,i like the new un-do button GET BACK GET BACK ...,1
88085,thx now i can b its ok re top sry thought cait...,0
88086,take t brb swian i was ogin b i was going blue...,1


In [4]:
def read_kaggle_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["Text","oh_label"]]

kaggle_data = read_kaggle_data("kaggle_parsed_dataset.csv")
kaggle_data = kaggle_data.rename(columns={"oh_label": "label", "Text": "message"})
kaggle_data

,message,label
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
...,...,...
8794,"""Never really gave it much thought. I just fig...",0
8795,"""Nadie se salva de la regla 34 xd""",0
8796,"""Question: Are you a boy or a girl?""",0
8797,"""Leave your email or phone number and maybe yo...",1


In [5]:
from sklearn.model_selection import train_test_split

league_train, league_eval_and_test = train_test_split(league_data, train_size=0.8, random_state = 4)
league_eval, league_test = train_test_split(league_eval_and_test, train_size=0.5, random_state = 4)
league_train.reset_index(inplace=True, drop=True)
league_eval.reset_index(inplace=True, drop=True)
league_test.reset_index(inplace=True, drop=True)

print(f'league train rows: {len(league_train.index):,}')
print(f'league eval rows: {len(league_eval.index):,}')
print(f'league test rows: {len(league_test.index):,}')

league train rows: 70,470
league eval rows: 8,809
league test rows: 8,809


In [13]:
labels=["0","1"]

train_data = pd.concat([kaggle_data,league_train])
train_data

,message,label
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
...,...,...
70465,No no zed I forgot to report you last game Don...,1
70466,comin bot soon wards? k ryze no flash u need b...,0
70467,mid and akali top that'd be more logical whate...,0
70468,trap ard gj you are 3 vs 3 gg my nob team nida...,1


In [14]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_data)
ds['validation'] = Dataset.from_pandas(league_eval)
ds['test'] = Dataset.from_pandas(league_test)
ds

DatasetDict({
    train: Dataset({
        features: ['message', 'label', '__index_level_0__'],
        num_rows: 79269
    })
    validation: Dataset({
        features: ['message', 'label'],
        num_rows: 8809
    })
    test: Dataset({
        features: ['message', 'label'],
        num_rows: 8809
    })
})

## Tokenizing

Tokenize the texts:

In [15]:
from transformers import AutoTokenizer

transformer_name = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

In [16]:
def tokenize(examples):
    return tokenizer(examples['message'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/79269 [00:00<?, ? examples/s]

Map:   0%|          | 0/8809 [00:00<?, ? examples/s]

,label,__index_level_0__,input_ids,attention_mask
0,1,0,"[0, 113, 1185, 26536, 110, 4252, 72, 2]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,0,1,"[0, 113, 118, 269, 218, 75, 1346, 110, 477, 4,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,0,2,"[0, 113, 250, 48669, 45421, 176, 48669, 43409,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,0,3,"[0, 113, 8458, 225, 114, 47, 33976, 23126, 120...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,0,4,"[0, 113, 347, 37457, 42336, 134, 438, 741, 374...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
79264,1,70465,"[0, 3084, 117, 992, 196, 38, 18774, 7, 266, 47...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
79265,0,70466,"[0, 175, 179, 14084, 1010, 24725, 116, 449, 91...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
79266,0,70467,"[0, 16079, 8, 18735, 3644, 299, 14, 1017, 28, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
79267,1,70468,"[0, 41970, 24823, 821, 267, 47, 32, 155, 1954,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Model Creation and Training

Create the transformer model:

In [17]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [18]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    num_labels=len(labels)
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config).to(device)
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

Create the trainer object and train:

In [19]:
from transformers import TrainingArguments

num_epochs = 2
batch_size = 48
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    weight_decay=weight_decay,
    fp16=True
)

In [20]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [21]:
trainer.save_model("league_cyberb-ROBERTA")

## Testing

### Testing on League Data

Evaluate on the test partition:

In [22]:
test_ds = ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds.to_pandas()

Map:   0%|          | 0/9689 [00:00<?, ? examples/s]

,label,input_ids,attention_mask
0,0,"[0, 6149, 2]","[1, 1, 1]"
1,1,"[0, 4321, 6620, 2968, 29, 55, 6620, 828, 5559,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,0,"[0, 113, 37457, 43409, 288, 26644, 47, 581, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[0, 10010, 74, 34425, 939, 206, 51, 8052, 1275...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[0, 113, 30250, 324, 47, 32, 10, 35488, 22, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...
9684,1,"[0, 261, 4173, 116, 384, 4, 139, 1021, 22984, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9685,1,"[0, 2527, 475, 17202, 1717, 236, 1084, 939, 64...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9686,0,"[0, 113, 1106, 209, 27726, 856, 15291, 1368, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9687,1,"[0, 36146, 748, 34090, 266, 17487, 14223, 117,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [23]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[ 3.421875  , -3.234375  ],
       [-1.3349609 ,  0.9951172 ],
       [-0.39624023,  0.2421875 ],
       ...,
       [ 0.9892578 , -0.9477539 ],
       [-4.2460938 ,  3.2753906 ],
       [-4.4453125 ,  3.4160156 ]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 1]), metrics={'test_loss': 0.20473513007164001, 'test_accuracy': 0.9100010320982558, 'test_runtime': 80.2246, 'test_samples_per_second': 120.773, 'test_steps_per_second': 2.518})

In [24]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.89      0.91      0.90      4424
       toxic       0.93      0.91      0.92      5265

    accuracy                           0.91      9689
   macro avg       0.91      0.91      0.91      9689
weighted avg       0.91      0.91      0.91      9689



### Testing on Dota Data

In [ ]:
def read_dota_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["text","target"]]

dota_data = read_dota_data('tagged-data.csv')
dota_data = dota_data.rename(columns={"text": "message", "target": "label"})
dota_data['label'] = dota_data['label'].replace(2,1)
dota_data

In [70]:
dota_ds = DatasetDict()
dota_ds['test'] = Dataset.from_pandas(dota_data)
dota_ds

DatasetDict({
    test: Dataset({
        features: ['message', 'label'],
        num_rows: 3267
    })
})

In [71]:
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds_dota.to_pandas()

Map:   0%|          | 0/3267 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 4012, 3549, 2094, 2033, 5939, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 3374, 11265, 2595, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
2,0,"[101, 2054, 2003, 1996, 2190, 11350, 1029, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
3,0,"[101, 2158, 2008, 4223, 2006, 12946, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
4,0,"[101, 2025, 2746, 2046, 2377, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
...,...,...,...,...
3262,1,"[101, 1059, 2102, 1029, 1042, 1029, 2004, 1126...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3263,1,"[101, 2017, 2442, 2428, 11891, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
3264,1,"[101, 2017, 2031, 10041, 2447, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
3265,1,"[101, 3565, 10041, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"


In [72]:
output_dota = trainer.predict(test_ds_dota)
output_dota

PredictionOutput(predictions=array([[-1.2646484,  1.6171875],
       [ 4.3359375, -4.8046875],
       [-4.1445312,  4.9257812],
       ...,
       [-4.1601562,  5.0390625],
       [ 2.6445312, -2.53125  ],
       [-3.5527344,  4.3085938]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 2.7994821071624756, 'test_accuracy': 0.5677992041628406, 'test_runtime': 1.7236, 'test_samples_per_second': 1895.454, 'test_steps_per_second': 40.033})

In [ ]:
y_true = output_dota.label_ids
y_pred = np.argmax(output_dota.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))